# read

In [1]:
import pandas as pd; pd.set_option('display.max_rows', None)
import numpy as np; np.random.seed(0)
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('./data/train_df.csv', encoding='cp949')
test  = pd.read_csv('./data/test_df.csv', encoding='cp949')
sub   = pd.read_csv('./data/sample_submission.csv', encoding='cp949')

In [2]:
train.shape, test.shape, sub.shape

((32000, 6), (4640, 5), (4640, 2))

In [3]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9
3,3,1154500002014200,5011000315087400,식품,농산물,10
4,4,1165000021008300,5011000177051200,식품,가공식품,3


In [4]:
test.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM
0,32000,5013000043028400,1165000021097200,식품,농산물
1,32001,5013000044016100,1154500002066400,식품,농산물
2,32002,5013000205030200,4139000102013200,식품,농산물
3,32003,5013000205030200,4221000040093400,식품,농산물
4,32004,5013000268011400,2726000004017100,식품,농산물


# preprocessing

In [5]:
large_label = dict(train.groupby('DL_GD_LCLS_NM').mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)
mid_label   = dict(train.groupby('DL_GD_MCLS_NM').mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)

train['DL_GD_LCLS_NM'] = train['DL_GD_LCLS_NM'].map(large_label)
test['DL_GD_LCLS_NM']  = test['DL_GD_LCLS_NM'].map(large_label)

train['DL_GD_MCLS_NM'] = train['DL_GD_MCLS_NM'].map(mid_label)
test['DL_GD_MCLS_NM']  = test['DL_GD_MCLS_NM'].map(mid_label)

In [6]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,6.678694,3.672897,3
1,1,1135000009051200,5011000178037300,5.407921,4.444134,3
2,2,1135000030093100,5011000265091400,6.678694,10.375479,9
3,3,1154500002014200,5011000315087400,4.658195,4.297401,10
4,4,1165000021008300,5011000177051200,4.658195,6.807151,3


In [7]:
# 격자공간고유번호 슬라이싱

for i in [3, 4, 5, 6, 7, 8, 9]:
    train['send_' + str(i)] = train['SEND_SPG_INNB'].astype('str').str[:i].astype('int64')
    test['send_' + str(i)]  = test['SEND_SPG_INNB'].astype('str').str[:i].astype('int64')
    
    train['rec_' + str(i)] = train['REC_SPG_INNB'].astype('str').str[:i].astype('int64')
    test['rec_' + str(i)]  = test['REC_SPG_INNB'].astype('str').str[:i].astype('int64')

In [8]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,send_3,rec_3,send_4,rec_4,send_5,rec_5,send_6,rec_6,send_7,rec_7,send_8,rec_8,send_9,rec_9
0,0,1129000014045300,5011000220046300,6.678694,3.672897,3,112,501,1129,5011,11290,50110,112900,501100,1129000,5011000,11290000,50110002,112900001,501100022
1,1,1135000009051200,5011000178037300,5.407921,4.444134,3,113,501,1135,5011,11350,50110,113500,501100,1135000,5011000,11350000,50110001,113500000,501100017
2,2,1135000030093100,5011000265091400,6.678694,10.375479,9,113,501,1135,5011,11350,50110,113500,501100,1135000,5011000,11350000,50110002,113500003,501100026
3,3,1154500002014200,5011000315087400,4.658195,4.297401,10,115,501,1154,5011,11545,50110,115450,501100,1154500,5011000,11545000,50110003,115450000,501100031
4,4,1165000021008300,5011000177051200,4.658195,6.807151,3,116,501,1165,5011,11650,50110,116500,501100,1165000,5011000,11650000,50110001,116500002,501100017


In [9]:
train.columns[6:]

Index(['send_3', 'rec_3', 'send_4', 'rec_4', 'send_5', 'rec_5', 'send_6',
       'rec_6', 'send_7', 'rec_7', 'send_8', 'rec_8', 'send_9', 'rec_9'],
      dtype='object')

In [10]:
for col in list(train.columns[6:]):
    mapping_dict = dict(train.groupby(col).mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)
    train[col] = train[col].map(mapping_dict)
    test[col]  = test[col].map(mapping_dict)

In [11]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,send_3,rec_3,send_4,rec_4,send_5,rec_5,send_6,rec_6,send_7,rec_7,send_8,rec_8,send_9,rec_9
0,0,1129000014045300,5011000220046300,6.678694,3.672897,3,4.394558,5.806966,4.000000,5.939441,4.000000,5.939441,4.000000,5.939441,4.000000,5.934897,4.000000,6.333333,4.071429,4.500000
1,1,1135000009051200,5011000178037300,5.407921,4.444134,3,5.600000,5.806966,6.324324,5.939441,6.324324,5.939441,6.324324,5.939441,6.324324,5.934897,6.324324,5.334572,4.925926,5.418033
2,2,1135000030093100,5011000265091400,6.678694,10.375479,9,5.600000,5.806966,6.324324,5.939441,6.324324,5.939441,6.324324,5.939441,6.324324,5.934897,6.324324,6.333333,10.833333,5.972177
3,3,1154500002014200,5011000315087400,4.658195,4.297401,10,4.925743,5.806966,4.750000,5.939441,4.750000,5.939441,4.750000,5.939441,4.750000,5.934897,4.750000,6.153948,4.750000,6.991480
4,4,1165000021008300,5011000177051200,4.658195,6.807151,3,5.307692,5.806966,5.000000,5.939441,5.000000,5.939441,5.000000,5.939441,5.000000,5.934897,5.000000,5.334572,3.636364,5.418033


# encoding

In [12]:
train.drop(['index', 'SEND_SPG_INNB', 'REC_SPG_INNB'], axis=1, inplace=True)
test.drop(['index', 'SEND_SPG_INNB', 'REC_SPG_INNB'], axis=1, inplace=True)

In [16]:
train.drop(['send_3', 'rec_3', 'send_4', 'rec_4', 'send_6', 'rec_6', 'send_7', 'rec_7', 'send_8', 'rec_8'], axis=1, inplace=True)
test.drop(['send_3', 'rec_3', 'send_4', 'rec_4', 'send_6', 'rec_6', 'send_7', 'rec_7', 'send_8', 'rec_8'], axis=1, inplace=True)

In [17]:
train.head()

,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,send_5,rec_5,send_9,rec_9
0,6.678694,3.672897,3,4.000000,5.939441,4.071429,4.500000
1,5.407921,4.444134,3,6.324324,5.939441,4.925926,5.418033
2,6.678694,10.375479,9,6.324324,5.939441,10.833333,5.972177
3,4.658195,4.297401,10,4.750000,5.939441,4.750000,6.991480
4,4.658195,6.807151,3,5.000000,5.939441,3.636364,5.418033


# modeling

In [18]:
X = train.drop('INVC_CONT', axis=1)
y = train['INVC_CONT']

In [19]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

model = CatBoostRegressor(random_state=0, random_strength=0)

param = {
    
}

gscv = GridSearchCV(
    estimator=model, param_grid=param, scoring='neg_root_mean_squared_error', 
    refit=True, cv=5
)

gscv.fit(X, y)

Learning rate set to 0.068341
0:	learn: 5.3803609	total: 154ms	remaining: 2m 33s
1:	learn: 5.3235161	total: 164ms	remaining: 1m 21s
2:	learn: 5.2655785	total: 178ms	remaining: 59.2s
3:	learn: 5.2102683	total: 189ms	remaining: 47.1s
4:	learn: 5.1608535	total: 197ms	remaining: 39.2s
5:	learn: 5.1175366	total: 207ms	remaining: 34.3s
6:	learn: 5.0763948	total: 216ms	remaining: 30.7s
7:	learn: 5.0386952	total: 226ms	remaining: 28.1s
8:	learn: 5.0067985	total: 239ms	remaining: 26.3s
9:	learn: 4.9733210	total: 254ms	remaining: 25.2s
10:	learn: 4.9416746	total: 263ms	remaining: 23.6s
11:	learn: 4.9121195	total: 271ms	remaining: 22.3s
12:	learn: 4.8869442	total: 280ms	remaining: 21.3s
13:	learn: 4.8632223	total: 293ms	remaining: 20.6s
14:	learn: 4.8441103	total: 303ms	remaining: 19.9s
15:	learn: 4.8262729	total: 314ms	remaining: 19.3s
16:	learn: 4.8106594	total: 326ms	remaining: 18.9s
17:	learn: 4.7960707	total: 359ms	remaining: 19.6s
18:	learn: 4.7839844	total: 369ms	remaining: 19.1s
19:	learn

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x000001EC8672C460>,
             param_grid={}, scoring='neg_root_mean_squared_error')

In [20]:
print(gscv.best_params_)
print(-gscv.best_score_)

{}
5.031346651431191


In [21]:
pred = gscv.predict(test)
sub['INVC_CONT'] = pred

In [22]:
sub.head()

,index,INVC_CONT
0,32000,8.142843
1,32001,5.532692
2,32002,3.909332
3,32003,4.504638
4,32004,5.092477


In [23]:
# sub.to_csv('./sub/logistics_04.csv', index=False)